In [117]:
import pandas as pd

In [118]:
hic_y3_metadata = pd.read_csv('HPRC_Illumina_HiC_Metadata_HIC_Y3.csv')
hic_y3_transfer = pd.read_csv('HiC-Y3.transfer.csv',header=None)

In [119]:
hprc_production_biosample = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/hprc-production-biosample-table.tsv',sep='\t')

In [120]:
# title, data_type, library_layout, library_selection, shear_method
hic_y3_metadata = hic_y3_metadata[['filename', 'library_ID','library_strategy', 'library_source','platform', 'instrument_model','design_description',\
				 'generator_facility','generator_contact','assembly','data_type','filetype']]
# library_source, data type
hic_y3_metadata['library_source'] ='GENOMIC'
hic_y3_metadata['data_type'] = 'unaligned reads'
hic_y3_metadata['library_layout'] = 'paired'
hic_y3_metadata['library_strategy'] = 'Hi-C'
hic_y3_metadata['library_selection'] = 'RANDOM'
hic_y3_metadata['shear_method'] = 'no shear'
title_list = []
# hic_y3_metadata['title']
for data in hic_y3_metadata['library_ID'].tolist():
	title_list.append('Illumina Sequencing of Omni-C Libraries of ' + data.split('-')[0])
hic_y3_metadata['title'] = title_list

In [121]:
hic_y3_metadata = hic_y3_metadata[hic_y3_metadata['filename'].isin([file.split('/')[-1] for file in hic_y3_transfer[1].tolist()])]
hic_y3_metadata = hic_y3_metadata.sort_values(by='filename')


In [122]:
hic_y3_metadata['common_id'] = hic_y3_metadata['filename'].str.extract(r'(^.+_L\d{3})')

In [123]:
df_R1 = hic_y3_metadata[hic_y3_metadata['filename'].str.contains('_R1_')].copy()
df_R2 = hic_y3_metadata[hic_y3_metadata['filename'].str.contains('_R2_')].copy()
# Rename the filename column in the R2 DataFrame to filename2
df_R2 = df_R2.rename(columns={'filename': 'filename2'})
df_merged = pd.merge(df_R1, df_R2, on=[col for col in hic_y3_metadata.columns if col not in ['filename', 'filename2', 'merge_key']], suffixes=('_R1', '_R2'))
df_merged['library_ID'] = df_merged['common_id']
df_merged.drop(columns=['common_id'],inplace=True)

df_merged['Sample'] = [sample.split('-')[0] for sample in df_merged['filename'].tolist()]
hic_y3_sra = pd.merge(df_merged, hprc_production_biosample[['Sample','Accession']], on='Sample',how='inner')

In [124]:
len(hic_y3_sra['filename'].tolist() + hic_y3_sra['filename2'].tolist())

752

In [125]:
hic_y3_sra.drop(columns=['Sample'], inplace=True)
hic_y3_sra.rename(columns={'Accession':'biosample_accession'},inplace=True)

In [126]:
hic_y3_sra

,filename,library_ID,library_strategy,library_source,platform,instrument_model,design_description,generator_facility,generator_contact,assembly,data_type,filetype,library_layout,library_selection,shear_method,title,filename2,biosample_accession
0,HG00099-1_S20_L001_R1_001.fastq.gz,HG00099-1_S20_L001,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of HG0...,HG00099-1_S20_L001_R2_001.fastq.gz,SAMN33758778
1,HG00099-1_S20_L002_R1_001.fastq.gz,HG00099-1_S20_L002,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of HG0...,HG00099-1_S20_L002_R2_001.fastq.gz,SAMN33758778
2,HG00099-1_S20_L003_R1_001.fastq.gz,HG00099-1_S20_L003,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of HG0...,HG00099-1_S20_L003_R2_001.fastq.gz,SAMN33758778
3,HG00099-1_S20_L004_R1_001.fastq.gz,HG00099-1_S20_L004,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of HG0...,HG00099-1_S20_L004_R2_001.fastq.gz,SAMN33758778
4,HG00099-2_S28_L001_R1_001.fastq.gz,HG00099-2_S28_L001,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of HG0...,HG00099-2_S28_L001_R2_001.fastq.gz,SAMN33758778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,NA20905-1_S22_L004_R1_001.fastq.gz,NA20905-1_S22_L004,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of NA2...,NA20905-1_S22_L004_R2_001.fastq.gz,SAMN33758801
372,NA20905-2_S30_L001_R1_001.fastq.gz,NA20905-2_S30_L001,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of NA2...,NA20905-2_S30_L001_R2_001.fastq.gz,SAMN33758801
373,NA20905-2_S30_L002_R1_001.fastq.gz,NA20905-2_S30_L002,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of NA2...,NA20905-2_S30_L002_R2_001.fastq.gz,SAMN33758801
374,NA20905-2_S30_L003_R1_001.fastq.gz,NA20905-2_S30_L003,Hi-C,GENOMIC,ILLUMINA,Illumina NovaSeq 6000,Illumina Sequencing of Omni-C Libraries,University of California Santa Cruz,wseligma@ucsc.edu,unaligned,unaligned reads,fastq,paired,RANDOM,no shear,Illumina Sequencing of Omni-C Libraries of NA2...,NA20905-2_S30_L003_R2_001.fastq.gz,SAMN33758801


In [127]:
hic_y3_sra.to_csv('sra_biosample_HIC_Y3.tsv',index=False, sep='\t')